# 1. Install Package

In [1]:
!pip install mediapipe opencv-python

In [ ]:
# ! pip install mediapipe

# 2. Import Package

In [2]:
import cv2 # openCV 라이브러리 
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils # mediapipe에서 그림 그리기 관련 유틸리티 임포트
mp_pose = mp.solutions.pose # 포즈 인식 모듈 임포트

# 3. Calculate Angle

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) # 두 벡터 사이의 각도 구하기
    angle = np.abs(radians*180.0/np.pi) # 라디안을 각도로 변환
    
    if angle >180.0: # 각도가 180보다 크면 보정 / 팔은 360도 회전 x
        angle = 360-angle
        
    return angle 

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture("mp4/biceps curl.mp4")

count = 0
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # 프레임이 비어있는 경우 무시하고 다음 프레임으로 진행
        if not ret:
            continue
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # 어깨, 팔꿈치, 손목 간의 각도 계산
            angle = calculate_angle(shoulder, elbow, wrist)
            
            # 이미지에 계산된 각도 시각화
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                count += 1
                       
        except:
            pass
        
        # Render curl counter
        # angle 표시
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)      
        
        cv2.putText(image, str(count), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)  
        
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)  
        
        # 검출된 포즈 랜드마크를 이미지에 시각화
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        # 렌더링된 이미지를 Mediapipe Feed라는 창에 표시
        cv2.imshow('Mediapipe Feed', image)

        # 'q' 키를 누르면 루프 종료
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()